In [5]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
    # ソースコードを記述
    graph_builder = StateGraph(State)
    llm = ChatOpenAI(model=model_name)

    tool = TavilySearchResults(max_results=2)
    tools = [tool]
    llm_with_tools = llm.bind_tools(tools)

    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}
    
    graph_builder.add_node("chatbot", chatbot)

    tool_node = ToolNode(tools)
    graph_builder.add_node("tools", tool_node)
    graph_builder.add_conditional_edges("chatbot", tools_condition,)
    graph_builder.add_edge("tools", "chatbot")
    graph_builder.set_entry_point("chatbot")

    memory = MemorySaver()
    return graph_builder.compile(checkpointer=memory)

# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
    # ソースコードを記述
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values")
    for event in events:
        print(event["messages"][-1].content, flush=True)
    print()

# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini" 

# グラフの作成
# ソースコードを記述
graph = build_graph(MODEL_NAME)

# メインループ
# ソースコードを記述
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました!")
        break
    stream_graph_updates(graph, user_input)

こんにちは！
こんにちは！何かお手伝いできることがあれば教えてください。

1たす2は？
1たす2は3です。何か他に質問がありますか？

台湾観光について検索結果を教えて

[{"url": "https://travel.rakuten.co.jp/mytrip/ranking/sightseeing-taiwan", "content": "楽天トラベル\n\n宿・航空券・ツアー予約\n\n台湾人がおすすめする、台湾旅行で行くべき観光スポット41選\n\n目次\n\n台湾の基本情報\n\n台北市のおすすめ観光スポット\n\n台北市\n\n台北101\n\n台北のシンボル的存在とも言える「台北101」。名前の由来は地上101階であることからきており、ビルの中には、ショッピングモールやレストラン、オフィスがあります。\n高さ約509mにもなる超高層ビルの展望デッキからは、台北市内の景色が楽しめます。また、ライトアップも綺麗で遠くから眺めるのもおすすめです。\n\n台北市\n\n四四南村 （スースーナンツン）\n\n「台北101」から徒歩約5分にある「四四南村」は、レトロでおしゃれなエリアです。元々軍人村として使われていましたが、今はカフェや雑貨店として再利用されています。\n見どころは、「好丘（ハオチョウ）」というカフェ・雑貨店。Made in Taiwanの良質な食器や小物類などが売られているほか、カフェでは様々なベーグルを焼き上げており、マンゴーなど珍しい味も楽しめます。\n\n台北市\n\n士林夜市 [...] 【台湾人スタッフおすすめコメント】\n夕日の名所として知られています。\n\n高雄市\n\n高雄市立図書館新総館\n\n8階建てのおしゃれな図書館。シンプルなデザインですが、光の透過率が高く、緑に溢れており、静かで快適な空間となっています。\n屋上にある見晴らしの良い庭園からは、高雄85ビルや高雄港を一望できます。夜は、建物自体が美しくライトアップされます。\n\n高雄市\n\n旧鉄橋湿地教育園区\n\n川岸にある、サイクリングや自然などが楽しめるスポット。高屏大橋と旧鉄橋の間に位置する「旧鉄橋湿地公園」は、多数の人工池があり、様々な魚類や鳥類が生息しています。\n昔使われていた鉄橋の修復工事を行って完成した「旧鉄橋空中歩道」からは、周辺の湿地など豊か